<a href="https://colab.research.google.com/github/neonin04/ai-security/blob/SooIn/First%20Assignment/SooInHwang__transfer_learning_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

In [0]:
20184141 황수인


Transfer Learning Tutorial
==========================
**Author**: `Sasank Chilamkurthy <https://chsasank.github.io>`_

In this tutorial, you will learn how to train your network using
transfer learning. You can read more about the transfer learning at `cs231n
notes <http://cs231n.github.io/transfer-learning/>`__

Quoting these notes,

    In practice, very few people train an entire Convolutional Network
    from scratch (with random initialization), because it is relatively
    rare to have a dataset of sufficient size. Instead, it is common to
    pretrain a ConvNet on a very large dataset (e.g. ImageNet, which
    contains 1.2 million images with 1000 categories), and then use the
    ConvNet either as an initialization or a fixed feature extractor for
    the task of interest.

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initializaion, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.




In [0]:
#Transfer Learning(전이학습)이란 기존에 만들어진 모델과, 그 모델이 학습한 weights를 그대로 가져와 일부 레이어를 추가하여 학습하는 방법이다.
# = 마지막 레이어만 수정해서 본인의 목적에 맞게 다시 학습시키는 것
#여기서 사전에 학습이 완료된, 이미지 분류에 최적화 된 weights를 포함하는 모델을 Pre-Trained Model이라고 한다.

#여기서 할 것은 개미 와 벌 을 분류하는 모델을 학습하는 것!!

# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division
#파이썬 2와 3의 버전 차이로 인해 생기는 문제를 방지하고 호환이 되도록 하기 위해 사용하는 모듈
#괄호 안의 인자, 재지정 및 출력 분리하는 기능 불러오기


import torch #PyTorch 불러오기
import torch.nn as nn #모든 신경망 모듈의 기본 클래스 불러오기
import torch.optim as optim #다양한 최적화 알고리즘 구현시키는 라이브러리 불러오기
from torch.optim import lr_scheduler #torch.optim에서 학습률 관리 패키지 불러오기
import numpy as np #행렬이나 일반적으로 대규모 다차원 배열을 쉽게 처리 할 수 있도록 지원하는 파이썬의 라이브러리 불러오기
import torchvision #토치비전 패키지 불러오기
from torchvision import datasets, models, transforms #torchvision에서 데이터 세트, 모델 아키텍처, 이미지 변환 패키지 불러오기
import matplotlib.pyplot as plt #matplotlib 라이브러리에서 pyplot 모듈 불러오기 (시각화패키지)
import time #time 모듈 불러오기 (현재 시각을 반환)
import os #os 모듈 불러오기 (파일을 복사하거나 디렉터리를 생성하고 특정 디렉터리 내의 파일 목록을 구함)
import copy #copy 모듈 불러오기 (객체 사본에 대한 변경 사항은 원본 객체에 반영하는 역할)
#as **는 앞으로 셀에서 **를 약자로 사용할 것이라는 의미 (np는 numpy의 약자로 사용하겠다는 뜻)

plt.ion()   #대화형 모드 켜준다 (명령 하나를 입력하고 그 결과를 받는 방식 마치 대화를 주고 받는 것처럼)

Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [0]:
# 훈련을 위한 데이터셋을 불러오고 정규화하는 과정!
# 일단 검증을 위한 표준화만

#데이터는 전처리과정을 거쳐야 한다.
data_transforms = { #딕셔너리
    'train': transforms.Compose([ #키 값, 여러 transforms을 묶어서 편하게 사용할 수 있게하는 도구.
        transforms.RandomResizedCrop(224), #입력 이미지에서 임이 크기의 패치(224,224)를 추출. 데이터가 증가되는 부분이다.
        transforms.RandomHorizontalFlip(), #이미지 사이즈를 (224,224)를 가진 후, 이미지를 뒤집는 기능. 또 다른 데이터 증가된다.
        transforms.ToTensor(), #입력 이미지를 PyTorch 텐서로 변환.
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #입력 데이터 스케일링. 평균 및 표준 등의 값들은 데이터 세트에 대해 사전에 계산되어 있어야 한다. 
    ]),
    'val': transforms.Compose([
        transforms.Resize(256), #입력 이미지를 (256,256)으로 크기 재조정.
        transforms.CenterCrop(224), #입력 이미지 (224,224)의 중심부분을 자름.
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

#train, val에 맞는 dataloader 선언
#x는 ['train', 'val']의 각 value마다 x가 key값인 딕셔너리를 만들어낸다는 것
data_dir = 'hymenoptera_data' #데이터 파일 이름
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), #이미지가 배열되어 있는 일반 데이터 로더에 해당 OS 형식에 맞도록 입력 받은 경로를 연결.
                                          data_transforms[x])
                    for x in ['train', 'val']}  #['train']은 train set의 경로, ['val']은 val set의 경로
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, #불러온 data를 네트워크 입력으로 사용하기 위해 사전에 불러온다
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}
                  
                                              #batch_size:한번에 얼만큼의 데이터씩 진행할 것인지, 그 사이즈를 4로 정하고,
                                              #shuffle:모든 epoch마다 데이터를 뒤섞는다.
                                              #num_workers:data 로딩을 위해 몇 개의 서브 프로세스를 사용할 것인지를 4개로 결정한다.
              
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
#image_datasets 배열의 길이를 나타내는 변수 -> dataset_sizes['train']은 train set 사진 갯수, dataset_sizes['val']은 val set 사진 갯수.

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#torch.Tensor가 할당되는 장치를 나타내는 객체 #GPU가 이용 가능한지를 확인하는 것 -> CUDA 기기가 존재한다면, 아래 코드가 CUDA 장치를 출력

Visualize a few images 
Let's visualize a few training images so as to understand the data
augmentations.



In [0]:
#위에서 선언한 dataloader를 기반으로 실제 batch 단위로 visualize하는 과정 

def imshow(inp, title=None): #영상을 출력하는 함수
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0)) #넘파이 함수를 이용해 (1,2,0)을 전치
    mean = np.array([0.485, 0.456, 0.406]) #평균 배열 생성
    std = np.array([0.229, 0.224, 0.225]) #배열 생성
    inp = std * inp + mean #정규화를 해제
    inp = np.clip(inp, 0, 1) #배열의 값을 제한
    plt.imshow(inp) #결과를 화면에 출력
    if title is not None:
        plt.title(title) #출력한 결과 화면에 제목을 표시
    plt.pause(0.001)  #갱신이 될 때까지 잠시 기다림

inputs, classes = next(iter(dataloaders['train'])) #한개의 배치(batch)만큼 이미지를 불러온다. 위에서 배치 사이즈를 4로 했으니 사진 4장이 로드됨.

out = torchvision.utils.make_grid(inputs) #make_grid 함수를 통해 배치로부터 격자(그리드) 형태의 이미지를 만들고 보여줌.

imshow(out, title=[class_names[x] for x in classes]) #이미지를 출력

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [0]:
#모델을 학습하기 위한 일반 함수를 작성 (학습율(learning rate) 관리(scheduling), 최적의 모델 구하기)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25): #지정된 알고리즘을 사용하여 학습 된 모델을 만듬.(필요한 변수는 모델, 기준, 최적화기능, 관리기, num_epoch 이다)
#scheduler 매개변수는 LR 스케쥴러 객체로 torch.optim.lr_scheduler에서 나온 것  #num-epochs: batch가 데이터 전체를 다 도는 횟수의 제한을 의미
    since = time.time() #시작 시간을 기록한다 (분류하는데 걸리는 총 소요 시간을 계산하기 위해)

    best_model_wts = copy.deepcopy(model.state_dict()) #model.state_dict:각 계층을 매개변수 텐서로 매핑되는 파이썬 사전(dict) 객체의 모델을 깊은 복사 실행(내부의 객체들까지 모두 복사되는 것)  
    #깊은 복사하는 이유: 처음에 만들었던 사전(dict)객체와 복사된 객체가 전혀 달라지기 때문에 어느 한쪽을 수정한다고 해서 다른 한쪽이 영향 받는 일이 없게 하기 위해
    best_acc = 0.0 #최고정확도 변수

    for epoch in range(num_epochs): #epoch: 한번 모든 트레이닝 데이터에 대해서 forward pass(계산단계)와 backward pass(가중치의 변화를 세는 과정)를 진행 한 상태를 의미.
                                    #데이터셋을 수차례 반복.
        print('Epoch {}/{}'.format(epoch, num_epochs - 1)) #epoch을 카운트해서 그 값을 프린트(출력)한다 (.format은 앞에 프린트 해줄 {}자리에 ()안의 변수들을 차례로 넣어준다)
        print('-' * 10)
              
        #각 epoch은 학습 단계(train)와 검증 단계(validation)를 갖는다.
        for phase in ['train', 'val']: #train mode와 validation mode 순으로 진행
              
            if phase == 'train': #train 단계인 경우,
                scheduler.step() #학습 속도를 변경하고 초기 값을 유지한 채로
                model.train() #모델을 학습 모드로 설정하고
            else:
                model.eval() #validation 단계인 경우, 모델을 평가 모드로 설정한다.

            running_loss = 0.0 #손실값 변수
            running_corrects = 0 #정확도 변수

            for inputs, labels in dataloaders[phase]: #dataloader로부터 dataset과 그에 해당되는 label을 불러와서 반복한다
                inputs = inputs.to(device) #GPU로 입력데이터를 올림
                labels = labels.to(device) #GPU로 label을 올림

                optimizer.zero_grad() #매개변수 기울기를 0으로 설정. 왜냐하면 파이토치는 backward pass를 할 때 기울기를 계속 축적하는데 초기화를 안하면 최적값을 찾지 못할 수도 있다.

                #순전파
                #학습 시에만 연산 기록을 추적
                with torch.set_grad_enabled(phase == 'train'): #인수에 따라 기울기를 활성화 또는 비활성화 함.
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) #마지막 레이어의 모든 요소 중 최대 값을 반환해서 예측 값으로 지정
                    loss = criterion(outputs, labels)

                    
                    if phase == 'train': #학습 단계인 경우 (이 모드에서는 가중치를 업데이트 한다)
                        loss.backward() #역전파와
                        optimizer.step() #최적화를 한다 (대부분의 최적화기능이 지원하는 단순화된 버전)

                #변수들의 통계치를 구한다
                running_loss += loss.item() * inputs.size(0) #손실값의 아이템과 각 배치에 있는 총 이미지 수를 다 곱한 값이 손실값이 된다.
                running_corrects += torch.sum(preds == labels.data) #주어진 차수에서 입력 텐서의 각 행의 합을 반환

            epoch_loss = running_loss / dataset_sizes[phase] #손실값을 학습,검증에 사용 된 모든 예제들의 수로 나누면 한 에폭의 손실값이 된다.
            epoch_acc = running_corrects.double() / dataset_sizes[phase] #정확도 변수를 검증에 사용 된 모든 예제들의 수로 나누면 에폭의 정확도가 구해진다.

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc: #만약 val과 에폭정확도가 최고정확도 보다 크면,
                best_acc = epoch_acc #에폭정확도를 최고정확도로 바꾸고
                best_model_wts = copy.deepcopy(model.state_dict()) #모델을 깊은 복사한다.

        print()

    time_elapsed = time.time() - since #경과된 시간을 계산
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts) #가장 나은 모델 가중치를 불러옴
    return model #모델을 반환

Visualizing the model predictions
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Generic function to display predictions for a few images




In [0]:
#위에서 작성한 함수들을 이용해 모델 예측 시각화하기
#(위에서 겹치는 함수들은 쓰지 않았습니다.)

def visualize_model(model, num_images=6): #6개의 이미지로 모델 시각화해서,
    was_training = model.training #훈련된 모델을 was_training 변수에 넣는다
    model.eval()
    images_so_far = 0
    fig = plt.figure() #pyplot 모듈을 이용해 새로운 figure를 생성한다

    with torch.no_grad(): #검증 세트에서 모델의 성능을 평가하고 자동 그라데이션에서 기울기 추적을 비활성화시킨다.
        for i, (inputs, labels) in enumerate(dataloaders['val']): #리스트 'val'의 그 순서와 값을 전달, (enumerate:반복되는 구간에서 객체가 현재 어느 위치에 있는지 알려주는 인덱스 값이 필요할때 사용하는 함수)
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far) #그리드 속성을 설정한다.(세로3, 가로2, images_so_far자리에 그리기)
                ax.axis('off') #축 설정을 해제한 후
                ax.set_title('predicted: {}'.format(class_names[preds[j]])) #축의 이름을 설정한다.
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images: #만약 이미지의 개수가 images_so_far이면,
                    model.train(mode=was_training) #훈련된 모델을
                    return
        model.train(mode=was_training) #반환한다

Finetuning the convnet
----------------------

Load a pretrained model and reset final fully connected layer.




In [0]:
#미리 학습한 모델을 불러온 후 마지막의 완전히 연결된 계층을 초기화시키는 과정
#무작위 초기화 대신 ImageNet 데이터 세트에서 사전훈련된 네트워크로 네트워크를 초기화

model_ft = models.resnet18(pretrained=True) #pretrained 변수가 True인 경우, 가중치 값을 불러옴으로써 ImageNet에서 사전 훈련된 ResNet18 모델을 반환 
#(ImageNet의 ResNet시리즈가 torchvision 라이브러리에 구현되어 있다.)
num_ftrs = model_ft.fc.in_features #입력 노드의 갯수, ResNet18모델의 마지막 레이어에서 num_ftrs를 구해주는 함수
model_ft.fc = nn.Linear(num_ftrs, 2) #입력데이터(num_ftrs)에 대해 선형변환을 적용하는 함수
                                 #2:출력 노드 갯수는 2개가 된다. (여기서 Linear layer의 출력 노드 개수는 class의 개수와 같아야 하기 때문 -> 개미와 벌로 이루어진 2개의 class 분류)
#위 세줄은 전이학습의 핵심 중 핵심! pretrained-model의 마지막 레이어에 완전히 연결된 레이어(fully connected layer)를 추가로 삽입하고 노드를 연결시켜 주는 과정이다.

model_ft = model_ft.to(device) #GPU로 구현한 모델을 올림

criterion = nn.CrossEntropyLoss() #loss함수로 CrossEntropy를 사용 -> 불균형 훈련세트일 때 유용하게 사용된다.

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9) #모든 매개변수들이 최적화되었는지 관찰 (스토케스틱 경사 하강법을 쓴다), lr(learning rate,학습율)을 0.001로, 모멘텀을 0.9로 설정.
#momentum 쓰는 이유: SGD의 경우 전체 데이터 대신 일부 데이터의 모음에 대해서만 Loss 함수를 계산하고 기울기를 가중치 업데이트에 사용하기 때문에, lr이 작더라도 업뎃되는 가중치가 크다.
#  ->그래서 momentum을 써서 업뎃되는 가중치 비율을 상대적으로 더 줄임으로써 안정적으로 가중치를 업데이트 할 수 있다.

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) #각 단계에서 학습 속도를 계산하는 함수 (7 에폭마다 0.1씩 학습율 감소) #step_size:함수가 실행되는 횟수(에폭)

Train and evaluate
^^^^^^^^^^^^^^^^^^

It should take around 15-25 min on CPU. On GPU though, it takes less than a
minute.




In [0]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=25)

In [0]:
visualize_model(model_ft)

ConvNet as fixed feature extractor
----------------------------------

Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad == False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <http://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`__.




In [0]:
#완전히 연결된 최종 레이어(fully-connected layer)의 가중치를 제외한 모든 네트워크의 가중치를 고정
#마지막으로 완전히 연결된 레이어는 임의 가중치가 있는 새 레이어로 교체되며 이 레이어만 학습됨
#(위에서 겹치는 함수들은 쓰지 않았습니다.)

model_conv = torchvision.models.resnet18(pretrained=True) #torchvision 라이브러리에 정의되어 있는 ResNet18 모델 #위와 같이, pretrained 변수가 True인 경우 ImageNet에서 사전 훈련된 모델을 반환한다.
for param in model_conv.parameters():
    param.requires_grad = False #backward()를 실행할 때 기울기를 계산하지 않도록 파라미터를 고정. 학습속도가 빨라진다.

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device) #GPU로 모델을 올림

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9) #이전과 달리 최종 레이어의 매개변수만 최적화되었는지 관찰

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1) 

Train and evaluate
^^^^^^^^^^^^^^^^^^

On CPU this will take about half the time compared to previous scenario.
This is expected as gradients don't need to be computed for most of the
network. However, forward does need to be computed.




In [0]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

In [0]:
visualize_model(model_conv)

plt.ioff() #대화식 모드를 꺼주고
plt.show() #생성된 모든 figure를 보여준다

In [0]:
#reference
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
#https://pytorch.org/docs/stable/index.html